In [ ]:
# To do:
# Transfer readme to outline/make outline
# Make proj pres outline/find images
# Problem statement
# Start scraping

In [ ]:
# H1 Title 

In [ ]:
# H2 Data Collection, Cleaning, & EDA

In [ ]:
# Picture

In [ ]:
# H4 Problem statement

In [ ]:
# Problem Statement
# - Is it clear what the goal of the project is?
# - What type of model will be developed?
# - How will success be evaluated?
# - Is the scope of the project appropriate?
# - Is it clear who cares about this or why this is important to investigate?
# - Does the student consider the audience and the primary and secondary stakeholders?

In [ ]:
# H4 Data info

In [ ]:
# Data info and background 

In [ ]:
# H4  Subreddit 1 

In [ ]:
# Subreddit 1 info

In [ ]:
# H4 Subreddit 2 

In [ ]:
# Subreddit 2 info

In [ ]:
# Research questions

In [ ]:
# H2 Data Collection
# - Was enough data gathered to generate a significant result?
# - Was data collected that was useful and relevant to the project?
# - Was data collection and storage optimized through custom functions, pipelines, and/or automation?
# - Was thought given to the server receiving the requests such as considering number of requests per second?

#### imports

In [2]:
import requests
import pandas as pd
import json
import csv
import time
import datetime

#### testing API

In [ ]:
# Reddit's API allows 60 requests per minute. 

In [ ]:
# Here is the basic process to pull from the reddit API

In [3]:
url = 'https://api.pushshift.io/reddit/search/submission'

In [4]:
params = {
'subreddit': 'boardgames',
'size':100}

In [5]:
res = requests.get(url, params)
res.status_code

In [7]:
data=res.json()
posts = data['data']
len(posts)
df = pd.DataFrame(posts)

#### Checking available data

In [12]:
df.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext',
       'author_flair_template_id', 'author_flair_text',
       'author_flair_text_color', 'author_flair_type', 'author_fullname',
       'author_is_blocked', 'author_patreon_flair', 'author_premium',
       'awarders', 'can_mod_post', 'contest_mode', 'created_utc', 'domain',
       'full_link', 'gildings', 'id', 'is_created_from_ads_ui',
       'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscriber

In [13]:
df[['subreddit', 'selftext', 'title']].head()

,subreddit,selftext,title
0,boardgames,"For me, it was Gloomhaven. Looking back, the s...",What was the first game that really swept you ...
1,boardgames,"PSA: afaik, this is just Promo Packs 1 and 2 c...",Spirit Island: Feather &amp; Flame now availab...
2,boardgames,,Mariokart racing on Rainbowroad
3,boardgames,I've been pretty obsessed with C&amp;K the las...,I love Catan: Cities &amp; Knights. I'm lookin...
4,boardgames,Cole confirmed in the KS updates that an expan...,Oath expansion confirmed!


In [14]:
# get lots of comments quickly with https://www.reddit.com/r/pushshift/comments/newjr2/best_way_to_fetch_100k_or_more_comments/
# https://www.ergosum.co/how-to-use-the-reddit-api-with-pushshift-to-bypass-api-limits/

## Data Collection

To avoid converting timestamps and adding dates, I calculated the 'before' and 'after' parameters based off UTC time. 

It doesn't really matter that these are exact. These subreddits are active, so I found that a one-month span for around 100 posts worked well even if it didn't capture every post.

In [85]:
#Get today's date in UTC timestamp
from datetime import timezone
dt = datetime.datetime.now(timezone.utc)
utc_time = dt.replace(tzinfo=timezone.utc)
today = utc_time.timestamp()

print(today)    

# UTC datetime is clculated in "seconds elapsed". One year is about 31,536,000 seconds, one month is 2,628,288. 
onemonthish = 2620000

# Write a loop that creates a dictionary of before:after values to pass into my api function

rangelist={}
for i in range (0, 60):
    after = int(round(today - (onemonth*(i+1)),0)) 
    before = int(round(today - (onemonth*i), 0)) #When i is 0, "before" is set to today.
    rangelist[before]=after

1651117519.863908


Next I chose the fields I wanted to pull from the API, and wrote a function to automate api requests.

In [86]:
# Choosing the fields I want to pull from the API
fields = ['id', 'created_utc', 'title', 'selftext', 'author', 'upvote_ratio', 'score', 'num_comments', 'total_awards_received', 'is_created_from_ads_ui', 'url']

def query(subreddit, num, before, after):   
    params = {
    'subreddit': subreddit,
    'size':num,
    'before':before,
    'after':after,
    'fields':fields}    # defined above
    url = 'https://api.pushshift.io/reddit/search/submission/?'
    res = requests.get(url, params)

    data = res.json()
    test = pd.DataFrame(data['data'])
    return(test)

# Testing my function
query('learnpython', 1, 1651108665, 1648480377)

,author,created_utc,id,is_created_from_ads_ui,num_comments,score,selftext,title,total_awards_received,upvote_ratio,url
0,unit111,1648482436,tqc9ff,False,0,1,Basically title. I am trying to figure out if ...,Has anybody used Playwright with Behave,0,1.0,https://www.reddit.com/r/learnpython/comments/...


I then iterated over my before:after dictionary and passed key:value pairs into the api function to set my time range. 

In [87]:
# Pull and make dataframe for learnpython sub
# List comp version sometimes works and sometimes doesn't, possibly because I can't add sleep time. I've commented it out for now
# listdfs=[query('learnpython', 100, key, val) for key, val in rangelist.items()]

listdfs=[]
for key, val in rangelist.items():
    i = query('learnpython', 100, key, val) 
    print('pulled 100 rows')
    listdfs.append(i)
    time.sleep(5)

learnpython=pd.concat(listdfs)
learnpython.info()

pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
<class 'pandas.core.frame.DataFrame'>
In

In [88]:
# Pull and make dataframe for learnsql sub
# List comp version sometimes works and sometimes doesn't, possibly because I can't add sleep time. I've commented it out for now
# listdfs=[query('learnsql', 100, key, val) for key, val in rangelist.items()]

listdfs=[]
for key, val in rangelist.items():
    i = query('learnsql', 100, key, val) 
    print('pulled 100 rows')
    listdfs.append(i)
    time.sleep(5)

learnsql=pd.concat(listdfs)
learnsql.info()

pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
pulled 100 rows
<class 'pandas.core.frame.DataFrame'>
In

#### Export dataframes

In [89]:
# export df
learnpython.to_csv('learnpython2.csv')

In [90]:
learnsql.to_csv('learnsql2.csv')